In [1]:
# Importing main libraries

import jupyterthemes as jt
import numpy as np
import pandas as pd
from distfit import distfit
import scipy.stats
import yfinance as yf

In [2]:
# Choosing my favorite style
jt.jtplot.style(theme = "monokai", spines = "False")

In [ ]:
# Creating a random exponential distribution
expo_data = np.random.exponential(scale = 4, size = 100000)

In [5]:
# Set the VaR confidence level
confidence_level = 0.05

## Parametric Simulation

In [66]:
def  param_VaR_95(data):
    # Average daily return
    loc = np.mean(data)
    # Estimating the daily volatility
    scale = np.std(data)
    # Parametric VaR
    VaR_95 = scipy.stats.norm.ppf(confidence_level, loc, scale)
    return VaR_95

In [69]:
param_VaR_95(expo_data)

-2.538006781431142

## Historical Simulation (Simple)

In [70]:
# First way: the simplest you could get
def simple_VaR_95(data):
    return np.percentile(data, confidence_level*100)

In [71]:
simple_VaR_95(expo_data)

0.20689834305611907

## Historical Simulation (Bootstrap)

In [10]:
# This function will generate completely random samples by replacing data
def bootstrap(data):
    sample = np.random.choice(data, len(data))
    return np.percentile(sample, confidence_level*100)

In [11]:
bootstrap(expo_data)

0.20165123385106093

In [72]:
# This function will bootstrap over and over
def bs_sims(data, size = 1000):
    bs_replicates = [bootstrap(data) for i in range(size)]
    return bs_replicates

In [13]:
bs_sims(expo_data)

[0.2077648214646311,
 0.2088603431952828,
 0.20862126909168088,
 0.20734425873759063,
 0.20429204727855563,
 0.20870848815228954,
 0.20477731577567232,
 0.20824607137869086,
 0.20542161338339912,
 0.20389678826712743,
 0.20837891305037765,
 0.2110420154756138,
 0.20239272703191924,
 0.2069841261608775,
 0.20690587418184375,
 0.20475877851325458,
 0.20345649087363468,
 0.20305214850560557,
 0.20697268243620118,
 0.21008903889485048,
 0.20689145647501125,
 0.20419165173671422,
 0.2031690176141663,
 0.21106323991513942,
 0.2105840660687992,
 0.2074984880129557,
 0.20661632563049215,
 0.20077294850612523,
 0.20974787315138052,
 0.20850206215220385,
 0.20532324621993708,
 0.20914191793078912,
 0.2065900086908816,
 0.20616511676807134,
 0.2077648214646311,
 0.20794955017336098,
 0.20841705827273446,
 0.20850252350500428,
 0.2052722967166331,
 0.20734425873759063,
 0.20217985826407442,
 0.21151636271158014,
 0.20771399919195896,
 0.20427241316105998,
 0.20437190748363523,
 0.20261486299888992

In [14]:
# This function will average our bootstrap results
def non_param_VaR_95(data):
    VaR_95 = np.mean(bs_sims(data))
    return VaR_95

In [15]:
non_param_VaR_95(expo_data)

0.20648293135197576

## Monte Carlo Simulation

In [74]:
# This function is a Goodness of Fit Test using Komolgorov-Smirnov's method

def best_fit_df(data):
    # Initializing the model
    gof = distfit(stats = 'ks')
    
    # Applying
    gof.fit_transform(data)
    
    # Testing top 10 used fits(function's default argument)
    all_fits = gof.summary 
    
    # Selecting the best one
    bestfit_row = all_fits.iloc[0]
    bestfit = pd.DataFrame(bestfit_row).transpose()
    
    return bestfit

In [18]:
# What was the best fit?

def best_fit_str(data):
    best_distr = best_fit_df(data).iloc[0]
    perfect_distr = best_distr[0]
    return perfect_distr

In [23]:
best_fit_str(expo_data)

[distfit] >fit..
[distfit] >transform..
[distfit] >[norm      ] [0.00 sec] [ks: 2.8811490] [loc=3.999 scale=3.999]
[distfit] >[expon     ] [0.00 sec] [ks: 0.9492907] [loc=0.000 scale=3.999]
[distfit] >[pareto    ] [1.51 sec] [ks: 6.8375975] [loc=-1.651 scale=1.651]
[distfit] >[dweibull  ] [1.08 sec] [ks: 0.9492907] [loc=2.854 scale=2.806]
[distfit] >[t         ] [1.09 sec] [ks: 1.6633834] [loc=2.881 scale=2.358]
[distfit] >[genextreme] [3.14 sec] [ks: 4.0039570] [loc=1.832 scale=1.848]
[distfit] >[gamma     ] [0.75 sec] [ks: 0.9492907] [loc=0.000 scale=3.997]
[distfit] >[lognorm   ] [2.03 sec] [ks: 2.8811490] [loc=-0.296 scale=2.838]
[distfit] >[beta      ] [2.42 sec] [ks: 0.9492907] [loc=-0.097 scale=292348570701700.500]
[distfit] >[uniform   ] [0.00 sec] [ks: 15.4645191] [loc=0.000 scale=54.776]
[distfit] >[loggamma  ] [1.68 sec] [ks: 2.5474499] [loc=-1729.546 scale=219.142]
[distfit] >Compute confidence interval [parametric]


'expon'

In [185]:
monte_carlo_VaR_95(expo_data)

[distfit] >fit..
[distfit] >transform..
[distfit] >[norm      ] [0.00 sec] [ks: 2.8811490] [loc=3.993 scale=3.971]
[distfit] >[expon     ] [0.00 sec] [ks: 1.1688042] [loc=0.000 scale=3.993]
[distfit] >[pareto    ] [1.07 sec] [ks: 7.9736726] [loc=-1.615 scale=1.616]
[distfit] >[dweibull  ] [1.04 sec] [ks: 1.1688042] [loc=2.874 scale=2.804]
[distfit] >[t         ] [0.63 sec] [ks: 2.2334549] [loc=2.893 scale=2.366]
[distfit] >[genextreme] [3.02 sec] [ks: 4.8579897] [loc=1.836 scale=1.850]
[distfit] >[gamma     ] [0.44 sec] [ks: 1.1688042] [loc=0.000 scale=3.983]
[distfit] >[lognorm   ] [2.33 sec] [ks: 4.0039570] [loc=-0.301 scale=2.846]
[distfit] >[beta      ] [3.61 sec] [ks: 1.1688042] [loc=0.000 scale=419.196]
[distfit] >[uniform   ] [0.00 sec] [ks: 16.4235605] [loc=0.000 scale=59.788]
[distfit] >[loggamma  ] [2.28 sec] [ks: 2.8811490] [loc=-1193.200 scale=161.865]
[distfit] >Compute confidence interval [parametric]


0.2017175225314265

In [179]:
def monte_carlo_VaR_95(data):
    perfect_distr = best_fit_df(data).iloc[0]
    
    # Turning strings to reandom generation (based on given distribution) function
    random_func = eval('scipy.stats.' + perfect_distr[0] + '.rvs')
    
    # Adding loc, scale and eventual arguments
    try:
        random_distr = random_func(perfect_distr[5], loc = perfect_distr[3], scale = perfect_distr[4], size = 10000)
    except: 
        random_distr = random_func(loc = perfect_distr[3], scale = perfect_distr[4], size = 10000)

    VaR_95 = np.percentile(random_distr, confidence_level*100)
    return VaR_95

In [183]:
monte_carlo_VaR_95(expo_data)

[distfit] >fit..
[distfit] >transform..
[distfit] >[norm      ] [0.00 sec] [ks: 2.8811490] [loc=3.993 scale=3.971]
[distfit] >[expon     ] [0.00 sec] [ks: 1.1688042] [loc=0.000 scale=3.993]
[distfit] >[pareto    ] [1.09 sec] [ks: 7.9736726] [loc=-1.615 scale=1.616]
[distfit] >[dweibull  ] [0.98 sec] [ks: 1.1688042] [loc=2.874 scale=2.804]
[distfit] >[t         ] [0.49 sec] [ks: 2.2334549] [loc=2.893 scale=2.366]
[distfit] >[genextreme] [2.15 sec] [ks: 4.8579897] [loc=1.836 scale=1.850]
[distfit] >[gamma     ] [0.39 sec] [ks: 1.1688042] [loc=0.000 scale=3.983]
[distfit] >[lognorm   ] [2.06 sec] [ks: 4.0039570] [loc=-0.301 scale=2.846]
[distfit] >[beta      ] [3.27 sec] [ks: 1.1688042] [loc=0.000 scale=419.196]
[distfit] >[uniform   ] [0.00 sec] [ks: 16.4235605] [loc=0.000 scale=59.788]
[distfit] >[loggamma  ] [2.25 sec] [ks: 2.8811490] [loc=-1193.200 scale=161.865]
[distfit] >Compute confidence interval [parametric]


0.20853914857609085

## Let's Get Real

In [141]:
# Getting Stock adjusted close data
tick = 'BAC'
stock_data = yf.download(tickers = str(tick), time_period = 'all')['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [125]:
# Taking a better look
stock_data.describe()

count    10150.000000
mean        46.807058
std         73.891118
min          0.018525
25%          4.054294
50%         21.132608
75%         38.935505
max        416.179993
Name: Adj Close, dtype: float64

In [142]:
# Getting stock returns
stock_data['Returns'] = stock_data.pct_change()
returns = stock_data['Returns'][1:-1]

In [173]:
# VaR types compared (real data)
param_VaR_95(returns), simple_VaR_95(returns), non_param_VaR_95(returns), monte_carlo_VaR_95(returns)

[distfit] >fit..
[distfit] >transform..
[distfit] >[norm      ] [0.00 sec] [ks: 6.8375975] [loc=0.001 scale=0.024]
[distfit] >[expon     ] [0.0 sec] [ks: 15.4645191] [loc=-0.290 scale=0.290]
[distfit] >[pareto    ] [0.37 sec] [ks: 15.4645191] [loc=-47431.595 scale=47431.306]
[distfit] >[dweibull  ] [0.45 sec] [ks: 2.8811490] [loc=0.000 scale=0.012]
[distfit] >[t         ] [0.07 sec] [ks: 0.2606768] [loc=0.000 scale=0.012]
[distfit] >[genextreme] [0.22 sec] [ks: 2.5474499] [loc=0.001 scale=0.037]
[distfit] >[gamma     ] [0.15 sec] [ks: 6.3072307] [loc=-0.326 scale=0.002]
[distfit] >[lognorm   ] [0.24 sec] [ks: 6.8375975] [loc=-1.752 scale=1.752]
[distfit] >[beta      ] [0.33 sec] [ks: 6.8375975] [loc=-1.287 scale=11.820]
[distfit] >[uniform   ] [0.00 sec] [ks: 18.4984988] [loc=-0.290 scale=0.642]
[distfit] >[loggamma  ] [0.22 sec] [ks: 6.8375975] [loc=-4.995 scale=0.736]
[distfit] >Compute confidence interval [parametric]


(-0.03911709636932843,
 -0.03149354052497218,
 -0.03151444018025618,
 -0.030699088993917035)

In [184]:
# VaR types compared (pseudo data). Important: param vs others
param_VaR_95(expo_data), simple_VaR_95(expo_data), non_param_VaR_95(expo_data), monte_carlo_VaR_95(expo_data)

[distfit] >fit..
[distfit] >transform..
[distfit] >[norm      ] [0.00 sec] [ks: 2.8811490] [loc=3.993 scale=3.971]
[distfit] >[expon     ] [0.00 sec] [ks: 1.1688042] [loc=0.000 scale=3.993]
[distfit] >[pareto    ] [1.56 sec] [ks: 7.9736726] [loc=-1.615 scale=1.616]
[distfit] >[dweibull  ] [1.45 sec] [ks: 1.1688042] [loc=2.874 scale=2.804]
[distfit] >[t         ] [0.71 sec] [ks: 2.2334549] [loc=2.893 scale=2.366]
[distfit] >[genextreme] [2.93 sec] [ks: 4.8579897] [loc=1.836 scale=1.850]
[distfit] >[gamma     ] [0.57 sec] [ks: 1.1688042] [loc=0.000 scale=3.983]
[distfit] >[lognorm   ] [2.64 sec] [ks: 4.0039570] [loc=-0.301 scale=2.846]
[distfit] >[beta      ] [3.78 sec] [ks: 1.1688042] [loc=0.000 scale=419.196]
[distfit] >[uniform   ] [0.00 sec] [ks: 16.4235605] [loc=0.000 scale=59.788]
[distfit] >[loggamma  ] [2.40 sec] [ks: 2.8811490] [loc=-1193.200 scale=161.865]
[distfit] >Compute confidence interval [parametric]


(-2.538006781431142,
 0.20689834305611907,
 0.20668538154559696,
 0.2167247818814335)